In [1]:
// https://github.com/JWally/jsLPSolver
const solver = require('javascript-lp-solver')

In [2]:
const f_zip = (a, b) => a.map((k, i) => [k, b[i]]);

In [3]:
const f_zip_func = (a, b, func) => a.map((k, i) => func(k, b[i]));

In [4]:
const toObject = (map = new Map) =>
  Object.fromEntries
    ( Array.from
        ( map.entries()
        , ([ k, v ]) =>
            v instanceof Map
              ? [ k, toObject (v) ]
              : [ k, v ]
        )
    )


In [5]:
const input_data = require('./input_data');

## make structures for the mineral ids / names / prices

In [6]:
// make structures for the mineral ids / names / prices
function getMineralInfo(itemInfo, itemPrices, itemRequirements) {
    let tmp_set = new Set();
    let _discard1 = Object.keys(itemRequirements)
        .map(x => Object.keys(itemRequirements[x])
            .map(y => tmp_set.add(y)));

    let mineral_ids = Array.from(tmp_set);
    let mineral_names = Array.from(mineral_ids.map(x => itemInfo[x].n));
    let mineral_packaged_volumes = Array.from(mineral_ids.map(x => parseFloat(itemInfo[x].pv)));
    let mineral_prices = Array.from(mineral_ids.map(x => parseFloat(itemPrices[x])));

    return [mineral_ids, mineral_names, mineral_packaged_volumes, mineral_prices]
}

In [7]:
[mineral_ids, mineral_names, mineral_packaged_volumes, mineral_prices] = getMineralInfo(input_data.itemInfo, input_data.itemPrices, input_data.itemRequirements);

[
  [
    '34', '35',
    '36', '37',
    '38', '39',
    '40'
  ],
  [
    'Tritanium',
    'Pyerite',
    'Mexallon',
    'Isogen',
    'Nocxium',
    'Zydrine',
    'Megacyte'
  ],
  [
    0.01, 0.01,
    0.01, 0.01,
    0.01, 0.01,
    0.01
  ],
  [
     6.32, 17.5,
    137.7, 36.5,
     1159, 1335,
    627.8
  ]
]

## what are we requiring minerals for

In [8]:
let ship_build_map = new Map(Object.entries({ 16242: 100, 626: 1 }));
let ship_me_map = new Map(Object.entries({ 16242: 10, 626: 10 }));

console.log({ ship_build_map });
console.log({ ship_me_map });

{ ship_build_map: Map(2) { '626' => 1, '16242' => 100 } }
{ ship_me_map: Map(2) { '626' => 10, '16242' => 10 } }


## what minerals do we need to accomplish this

In [9]:
function calcRequiredMinerals(ship_map, me_map, mineral_ids, itemRequirements) {
    
    function calc_qty(count, quantity, me, fm) {
        return Math.max(parseInt(count),
            Math.ceil(Math.fround(parseInt(count) * parseInt(quantity) * (1.0 - parseInt(me) / 100.0) * parseFloat(fm)), 2));
    }

    var required_minerals_map = new Map(mineral_ids.map(x => [x, 0]));

    let _discard2 = ship_map.forEach((sbv, sbk, sbm) => {
        let ship_me = me_map.get(sbk);
        let ship_minerals_map = new Map(Object.entries(itemRequirements[sbk]));

        mineral_ids.map(mineral_id => {
            let smv = ship_minerals_map.get(mineral_id);
            smv = parseInt((smv === undefined) ? 0 : smv);
            let v = required_minerals_map.get(mineral_id);
            v = parseInt((v === undefined) ? 0 : v);
            required_minerals_map.set(mineral_id, v + calc_qty(sbv, smv, ship_me, 1.0));
        });
    });

   return [required_minerals_map];
}

In [10]:
[required_minerals_map] = calcRequiredMinerals(ship_build_map, ship_me_map, mineral_ids, input_data.itemRequirements);

console.log(f_zip(mineral_names, mineral_ids.map(x => required_minerals_map.get(x))));

[
  [ 'Tritanium', 4871630 ],
  [ 'Pyerite', 1153470 ],
  [ 'Mexallon', 394930 ],
  [ 'Isogen', 167230 ],
  [ 'Nocxium', 2701 ],
  [ 'Zydrine', 17381 ],
  [ 'Megacyte', 2661 ]
]


## What would these minerals cost if we bought them on the market?

In [11]:
minerals_cost = Math.ceil(f_zip_func(mineral_ids.map(x => required_minerals_map.get(x)),
                           mineral_prices, (a, b) => { return a * b }).reduce((a, b) => a + b));
console.log("minerals_cost: " + new Intl.NumberFormat().format(minerals_cost));


minerals_volume = Math.ceil(f_zip_func(mineral_ids.map(x => required_minerals_map.get(x)),
                           mineral_packaged_volumes, (a, b) => { return a * b }).reduce((a, b) => a + b));
console.log("minerals_volume: " + new Intl.NumberFormat().format(minerals_volume));



minerals_cost: 139,464,853
minerals_volume: 66,101


## make structures for the ore ids / yields / volumes / names / prices

In [12]:
function getOreInfo(efficiency, mineral_ids, itemInfo, itemPrices, itemYields) {

    let ore_ids = Object.keys(itemInfo)
        .filter(x => { t = itemInfo[x].n.split(/\s+/); return t.length == 2 && t[0] == 'Compressed' })
        .sort();

    let ore_yields = new Map(ore_ids.map(x => {
        ore_yield_map = new Map(Object.entries(itemYields[x]));
        return [x, new Map(mineral_ids.map(mineral_id => {
            oyv = ore_yield_map.get(mineral_id);
            oyv = parseInt((oyv === undefined) ? 0 : Math.floor(oyv * efficiency));
            return [mineral_id, oyv]
        }))]
    }));


    let ore_names = Array.from(ore_ids.map(x => itemInfo[x].n));
    let ore_packaged_volumes = Array.from(ore_ids.map(x => parseFloat(itemInfo[x].pv)));
    let ore_prices = Array.from(ore_ids.map(x => parseFloat(itemPrices[x])));


    return [ore_ids, ore_names, ore_packaged_volumes, ore_prices, ore_yields];
};

In [13]:
// we never get 100% of the yield when refining ore. 
refining_efficiency = parseFloat(0.50);

[ore_ids, ore_names, ore_packaged_volumes, ore_prices, ore_yields] = getOreInfo(refining_efficiency, mineral_ids, input_data.itemInfo, input_data.itemPrices, input_data.itemYields);

[
  [
    '28367', '28388',
    '28391', '28397',
    '28401', '28403',
    '28406', '28410',
    '28416', '28420',
    '28422', '28424',
    '28429', '28432'
  ],
  [
    'Compressed Arkonor',
    'Compressed Bistot',
    'Compressed Crokite',
    'Compressed Gneiss',
    'Compressed Hedbergite',
    'Compressed Hemorphite',
    'Compressed Jaspet',
    'Compressed Kernite',
    'Compressed Omber',
    'Compressed Spodumain',
    'Compressed Plagioclase',
    'Compressed Pyroxeres',
    'Compressed Scordite',
    'Compressed Veldspar'
  ],
  [
     8.8,  4.4, 7.81,  1.8,
    0.47, 0.86, 0.15, 0.19,
     0.3,   28, 0.15, 0.16,
    0.19, 0.15
  ],
  [
     368500, 331800, 953400,
     330700, 249900, 198900,
     109400,  26500,  15180,
    1217000,   7890,   5345,
       2367,   1902
  ],
  Map(14) {
    '28367' => Map(7) {
      '34' => 0,
      '35' => 1600,
      '36' => 600,
      '37' => 0,
      '38' => 0,
      '39' => 0,
      '40' => 60
    },
    '28388' => Map(7) {
      '34

## Solve for cheapest combination of ore to get required minerals

In [14]:
function calcRequiredOres(required_minerals_map, mineral_ids, mineral_names, ore_ids, ore_names, ore_prices, ore_yields) {

    // See https://github.com/JWally/jsLPSolver for examples
    
    let opt_vars = new Map(ore_ids.map((ore_id, i) => {
        let m = new Map(mineral_ids.map((mineral_id, j) => {
            return [mineral_names[j], ore_yields.get(ore_id).get(mineral_id)]
        }));
        m.set('_price', ore_prices[i].valueOf());
        return [ore_id, m];
    }));

    let opt_cons = new Map(f_zip(mineral_names, mineral_ids.map(x => new Map([['min', required_minerals_map.get(x).valueOf()]]))));

    // solver very slow with many integer contraints.
    // constrain the highest-priced ores so the resulting price differential will be smallest.
    let solver_goes_nuts_maxint = 10;
    let ore_idx_price = f_zip(ore_ids, ore_prices).sort((a, b) => b[1] - a[1]).map(x => x[0])
    let opt_ints = new Map(ore_idx_price.map((x, i) => [x, i<solver_goes_nuts_maxint?1:0]));

    let opt_model = toObject(new Map([
        ['optimize', '_price'],
        ['opType', 'min'],
        ['constraints', opt_cons],
        ['variables', opt_vars],
        ['ints', opt_ints],
        ]));

    let opt_res = solver.Solve(opt_model);
    
    let required_ores_map = new Map(Object.entries(opt_res)
                        .filter(x => ore_ids.indexOf(x[0]) >= 0)
                        .map(x => [x[0], Math.ceil(x[1])]));
    ore_ids.map(x => {
            if (required_ores_map.get(x) === undefined) {
                required_ores_map.set(x, 0);
            }
    });

    return [required_ores_map];
}

In [15]:
[required_ores_map] = calcRequiredOres(required_minerals_map, mineral_ids, mineral_names, ore_ids, ore_names, ore_prices, ore_yields);

console.log({required_ores_map});
console.log(f_zip(ore_names, ore_ids.map(x => required_ores_map.get(x))));

{
  required_ores_map: Map(14) {
    '28367' => 45,
    '28388' => 218,
    '28391' => 7,
    '28410' => 2786,
    '28416' => 2,
    '28422' => 4188,
    '28429' => 16218,
    '28432' => 16456,
    '28397' => 0,
    '28401' => 0,
    '28403' => 0,
    '28406' => 0,
    '28420' => 0,
    '28424' => 0
  }
}
[
  [ 'Compressed Arkonor', 45 ],
  [ 'Compressed Bistot', 218 ],
  [ 'Compressed Crokite', 7 ],
  [ 'Compressed Gneiss', 0 ],
  [ 'Compressed Hedbergite', 0 ],
  [ 'Compressed Hemorphite', 0 ],
  [ 'Compressed Jaspet', 0 ],
  [ 'Compressed Kernite', 2786 ],
  [ 'Compressed Omber', 2 ],
  [ 'Compressed Spodumain', 0 ],
  [ 'Compressed Plagioclase', 4188 ],
  [ 'Compressed Pyroxeres', 0 ],
  [ 'Compressed Scordite', 16218 ],
  [ 'Compressed Veldspar', 16456 ]
]


## How much will this ore cost?

In [16]:
minerals_cost = Math.ceil(f_zip_func(ore_ids
    .map((x, i) => required_ores_map.get(x)), ore_prices, (a, b) => { return a * b })
    .reduce((a, b) => a + b));
console.log("minerals_cost: " + new Intl.NumberFormat().format(minerals_cost));

minerals_volume = Math.ceil(f_zip_func(ore_ids
    .map((x, i) => required_ores_map.get(x)), ore_packaged_volumes, (a, b) => { return a * b })
    .reduce((a, b) => a + b));
console.log("minerals_volume: " + new Intl.NumberFormat().format(minerals_volume));


minerals_cost: 272,178,698
minerals_volume: 8,118


## What actual minerals and residuals (assuming refining_yield) do we expect?

In [17]:
function calcActualResidualMinerals(required_ores_map, mineral_ids, ore_ids, ore_yields) {
    let actual_minerals_map = new Map(mineral_ids.map(x => [x, 0]));

    ore_ids.map((x, i) => {
        required_qty = required_ores_map.get(x);
        required_qty = parseInt((required_qty === undefined) ? 0 : required_qty);

        yield_map = ore_yields.get(x);
        mineral_ids.map((y, j) => {
            myv = yield_map.get(y);
            myv = parseInt((myv === undefined) ? 0 : myv);
            amv = actual_minerals_map.get(y);
            amv = parseInt((amv === undefined) ? 0 : amv);
            actual_minerals_map.set(y, amv + required_qty * myv);

        })
    })

    let residual_minerals_map = new Map(f_zip(mineral_ids,
                                f_zip_func(mineral_ids.map(x => actual_minerals_map.get(x)),
                                           mineral_ids.map(x => required_minerals_map.get(x)),
                                           (a, b) => a - b)));

    return [actual_minerals_map, residual_minerals_map];
}

In [18]:
[actual_minerals_map, residual_minerals_map] = calcActualResidualMinerals(required_ores_map, mineral_ids, ore_ids, ore_yields);

console.log(f_zip(mineral_names, mineral_ids.map(x => actual_minerals_map.get(x))));
console.log(f_zip(mineral_names, mineral_ids.map(x => residual_minerals_map.get(x))));

[
  [ 'Tritanium', 4871906 ],
  [ 'Pyerite', 1153500 ],
  [ 'Mexallon', 394960 ],
  [ 'Isogen', 167234 ],
  [ 'Nocxium', 2800 ],
  [ 'Zydrine', 17440 ],
  [ 'Megacyte', 2700 ]
]
[
  [ 'Tritanium', 276 ],
  [ 'Pyerite', 30 ],
  [ 'Mexallon', 30 ],
  [ 'Isogen', 4 ],
  [ 'Nocxium', 99 ],
  [ 'Zydrine', 59 ],
  [ 'Megacyte', 39 ]
]
